In [1]:
#import gensim.function

text=open('Стихи.txt','r').readlines()
text = [i for i in text if i != '' and 'Стихотворения' not in i]

In [2]:
text = [i.strip() for i in text]

In [3]:
text = [i for i in text if i != '' and 'Стихотворения' not in i]
text = text[:len(text)-243]
data=''.join(text)

In [4]:
len(data)

603368

In [5]:
data = data.replace('\xa0',' ')

for number in '123456789':
    data=data.replace(number,'0')
for english_letter in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
    data=data.replace(english_letter, '')
for english_letter in 'abcdefghijklmnopqrstuvwzyz':
    data=data.replace(english_letter, '')    

In [6]:
b=set(data)
c=dict()
for symbol in b:
    c[symbol] = data.count(symbol)
import operator
c1 = sorted(c.items(), key=operator.itemgetter(1))

In [8]:
Alphabet =''.join(c.keys())
Alphabet
#We don't delete any of the chars as now

'<мз;И[-„бтюМй)О–,г"(н\'еК«цсы…БФ/xвлЧ#_оПХхУЗШЦипэЩЛаЮшВ?яС А&ьфГЖ:Э>*.р]Е0РЯ»учДТщ—НкжЙёъд!'

In [66]:
return_first=True
def string_vectorizer(string, alphabet=Alphabet):
    vector = [[0 if char != letter else 1 for char in Alphabet] 
                  for letter in string]
    return vector
def vector_stringizer(vector, alphabet = Alphabet):
    answ=''
    if return_first:
        index = vector.argsort()[-1]
        answ+=Alphabet[index]
    else:
        index = vector.argsort()[-2]
        answ+=Alphabet[index]
    return answ

In [34]:

data_onehot = string_vectorizer(data)

In [10]:
import keras
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
#from tqdm import tqdm
from scipy import sparse

C:\Users\DK\Anaconda3.1\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [77]:
window_len = 15
X_total,Y_total=[],[]
i=1000
X_total.append(data_onehot[i-window_len:i])
Y_total.append(data_onehot[i])

In [78]:
def make_dataset(data_onehot):
    X_total,Y_total=[],[]
    for i in range(window_len, len(data_onehot)):
        X_total.append(data_onehot[i-window_len:i])
        Y_total.append(data_onehot[i])
    return np.array(X_total), np.array(Y_total)
X_total, Y_total=make_dataset(data_onehot)

In [79]:
X_total.shape

(595977, 15, 91)

In [80]:
X_train,X_test,Y_train,Y_test = train_test_split(X_total, Y_total, test_size=0.15, random_state = 1)
del X_total
del  Y_total

In [81]:
from  keras.callbacks import EarlyStopping
callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='min')

In [83]:
model = Sequential()
model.add(LSTM(100, input_shape=(window_len, len(Alphabet)), return_sequences=True))
model.add(Flatten())
model.add(Dense(len(Alphabet),activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam')
model.summary()
model.fit(X_train,Y_train, epochs=20,validation_data = (X_test, Y_test), batch_size=10, verbose=1,callbacks=[callback])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 15, 100)           76800     
_________________________________________________________________
flatten_8 (Flatten)          (None, 1500)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 91)                136591    
Total params: 213,391
Trainable params: 213,391
Non-trainable params: 0
_________________________________________________________________
Train on 506580 samples, validate on 89397 samples
Epoch 1/20
506580/506580 [==============================] - 920s 2ms/step - loss: 0.0368 - val_loss: 0.0341
Epoch 2/20
506580/506580 [==============================] - 885s 2ms/step - loss: 0.0329 - val_loss: 0.0324
Epoch 3/20
506580/506580 [==============================] - 931s 2ms/step - loss: 0.0315 - val_loss: 0.0317
Epoch 4/20
506580/506580 

KeyboardInterrupt: 

In [85]:
model.save('Pushkin1.h5')  # creates a HDF5 file 'my_model.h5'
  # deletes the existing model

# returns a compiled model
# identical to the previous one
#model = load_model('Pushkin1.h5')

In [93]:
text = 'Уж небо осенью '
from copy import deepcopy
text1 = deepcopy(text)

In [94]:
for i in range(1000):
    text_array = np.array(string_vectorizer(text1)).reshape(1,window_len,91)
    predicted_vector = model.predict(text_array)
    predicted_char = vector_stringizer(predicted_vector[0])
    text = text+predicted_char
    text1 = text1[1:]+predicted_char
    if i%20==0:
        print('Predicted '+str(i)+' letters ')
print(text)    

Predicted 0 letters 
Predicted 20 letters 
Predicted 40 letters 
Predicted 60 letters 
Predicted 80 letters 
Predicted 100 letters 
Predicted 120 letters 
Predicted 140 letters 
Predicted 160 letters 
Predicted 180 letters 
Predicted 200 letters 
Predicted 220 letters 
Predicted 240 letters 
Predicted 260 letters 
Predicted 280 letters 
Predicted 300 letters 
Predicted 320 letters 
Predicted 340 letters 
Predicted 360 letters 
Predicted 380 letters 
Predicted 400 letters 
Predicted 420 letters 
Predicted 440 letters 
Predicted 460 letters 
Predicted 480 letters 
Predicted 500 letters 
Predicted 520 letters 
Predicted 540 letters 
Predicted 560 letters 
Predicted 580 letters 
Predicted 600 letters 
Predicted 620 letters 
Predicted 640 letters 
Predicted 660 letters 
Predicted 680 letters 
Predicted 700 letters 
Predicted 720 letters 
Predicted 740 letters 
Predicted 760 letters 
Predicted 780 letters 
Predicted 800 letters 
Predicted 820 letters 
Predicted 840 letters 
Predicted 860 let

In [88]:
text1

'юблю тебя петра'

22

array([[4.68948383e-06, 1.32726545e-05, 1.82559847e-06, 5.59600792e-08,
        2.20814101e-07, 7.16160597e-09, 2.55149729e-07, 4.22420143e-08,
        4.58443792e-05, 6.15662866e-05, 9.85079751e-05, 1.48894244e-07,
        3.93421760e-06, 3.15374322e-08, 3.22220359e-07, 5.34448681e-08,
        7.25990731e-07, 4.61424497e-06, 5.46918208e-08, 8.29758804e-08,
        2.10189231e-04, 1.31498927e-08, 5.75646199e-02, 2.25692972e-07,
        6.55047172e-09, 2.34680101e-06, 1.94769200e-06, 4.35431662e-04,
        1.94357000e-07, 1.25667754e-07, 6.80677985e-08, 2.21273226e-07,
        4.01492422e-08, 1.18674041e-04, 4.41178243e-04, 1.91478033e-08,
        1.08824838e-09, 2.18117748e-07, 1.45933572e-02, 4.10452827e-07,
        4.09753333e-08, 6.03189983e-06, 1.65799037e-07, 3.21344977e-08,
        1.38635405e-08, 3.26991163e-08, 1.05397692e-02, 4.81374354e-06,
        1.13914083e-07, 5.94952780e-08, 7.33801713e-08, 2.23718937e-02,
        7.89767469e-08, 1.57684171e-05, 1.66292452e-07, 1.644149

'у'

'у'